# Test ``solve_tilda_parameters`` function

This is the older vesion of the function ``find_tilda_parameters``, where sigma_sum is defined. 

In [2]:
from simulation.mean_cv_autocorr_v2 import _solve_tilda_parameters
import matplotlib.pyplot as plt
from stats.mean import calculate_mean_from_params
from stats.variance import calculate_variance_from_params
from stats.cv import calculate_cv_from_params
from stats.fano_factor import calculate_fano_factor_from_params
from stats.autocorrelation import calculate_ac_from_params
import numpy as np
from simulation.simulate_telegraph_model import simulate_one_telegraph_model_system
from stats.mean import calculate_mean
from stats.variance import calculate_variance
from stats.cv import calculate_cv
from stats.autocorrelation import calculate_autocorrelation, calculate_ac_time_interp1d
%load_ext autoreload
%autoreload 2

## Test Case 1: Check against the computed parameters

In [12]:
mu_target = 15
t_ac_target = 1
cv_target = 1
sigma_sum =1 
# get the parameters using the scaled equations
rho,  d, sigma_b, sigma_u = _solve_tilda_parameters(sigma_sum, mu_target, t_ac_target, cv_target)
print('Parameters found:', rho,  d, sigma_b, sigma_u)

Parameters found: 449.0000000000068 15.000000000000236 0.5011135857461028 0.4988864142538973


In [ ]:
mu_target = 200
t_ac_target = 10
cv_target = 2
sigma_sum = 1

# get the parameters using the scaled equations
rho,  d, sigma_b, sigma_u = _solve_tilda_parameters(sigma_sum, mu_target, t_ac_target, cv_target)
print('Parameters found:', rho,  d, sigma_b, sigma_u)

# plug back in: 
mu_est   = calculate_mean_from_params(rho, d, sigma_b, sigma_u)
var_est  = calculate_variance_from_params(rho, d, sigma_b, sigma_u)
cv_est   = calculate_cv_from_params(rho, d, sigma_b, sigma_u)
ac_est   = calculate_ac_from_params(rho, d, sigma_b, sigma_u, t_ac_target)

# print comparison:
print(f"Mean:   target={mu_target:.4f},   analytical={mu_est:.4f},   error={100*(mu_est-mu_target)/mu_target:.2f}%")
print(f"CV:     target={cv_target:.4f},    analytical={cv_est:.4f},    error={100*(cv_est-cv_target)/cv_target:.2f}%")
print(f"AC({t_ac_target!r}): target={np.exp(-1):.4f}, analytical={ac_est:.4f}, error={100*(ac_est-np.exp(-1))/np.exp(-1):.2f}%")


Parameters found: 910.7335674386071 0.1118454128514586 0.024561609860503607 0.9754383901394963
Mean:   target=200.0000,   analytical=200.0000,   error=0.00%
CV:     target=2.0000,    analytical=2.0000,    error=0.00%
AC(10): target=0.3679, analytical=0.3679, error=0.00%


In [ ]:
mu_target = np.logspace(0, 4, 10)  # 1 to 10000
t_ac_target = np.logspace(-0.3, 2, 10)  # ~0.5 to 100
cv_target = np.logspace(-0.3, 0.7, 20)  # ~0.5 to 5
sigma_sum =1
# store the relative error for plotting later
error_data = {
    # 'sigma_sum': [],
    'mean_error': [],
    'cv_error': [],
    'ac_error': [],
    'mu': [],
    't_ac': [],
    'cv': []
}

success_data = {
    'success_count':0,
    'failure_count':0,
    'total_combo_count':len(mu_target) * len(t_ac_target) * len(cv_target)
}

# loop through every combination
for mu in mu_target:
    for t_ac in t_ac_target:
        for cv in cv_target:
            # get the parameters using the scaled equations
            try:
                rho,  d, sigma_b, sigma_u = _solve_tilda_parameters(sigma_sum, mu, t_ac, cv,
                                                                #   check_biological=False
                                                                  )
                success_data['success_count'] += 1
                
                print(f"Testing with mu={mu}, t_ac={t_ac}, cv={cv}")
                print('Parameters found:', rho,  d, sigma_b, sigma_u)

                # plug back in:
                mu_est   = calculate_mean_from_params(rho, d, sigma_b, sigma_u)
                var_est  = calculate_variance_from_params(rho, d, sigma_b, sigma_u)
                cv_est   = calculate_cv_from_params(rho, d, sigma_b, sigma_u)
                ac_est   = calculate_ac_from_params(rho, d, sigma_b, sigma_u, t_ac)
                
                # calculate errors
                mean_error = abs(100*(mu_est-mu)/mu)
                cv_error = abs(100*(cv_est-cv)/cv)
                ac_error = abs(100*(ac_est-np.exp(-1))/np.exp(-1))
                
                # store data
                error_data['mean_error'].append(mean_error)
                error_data['cv_error'].append(cv_error)
                error_data['ac_error'].append(ac_error)
                error_data['mu'].append(mu)
                error_data['t_ac'].append(t_ac)
                error_data['cv'].append(cv)

                # print comparison:
                print(f"Mean:   target={mu:.4f},   analytic={mu_est:.4f},   error={mean_error:.2f}%")
                print(f"CV:     target={cv:.4f},    analytic={cv_est:.4f},    error={cv_error:.2f}%")
                print(f"AC({t_ac!r}): target={np.exp(-1):.4f}, analytic={ac_est:.4f}, error={ac_error:.2f}%")
                
            except Exception as e:
                success_data['failure_count'] += 1
                print(f"FAILED: mu={mu}, t_ac={t_ac}, cv={cv} - Error: {e}")
                # Removed the 'break' statement - now continues to next combination

FAILED: mu=1.0, t_ac=0.5011872336272722, cv=0.5011872336272722 - Error: Invalid parameters: v = (mu_target * cv_target ** 2) - 1 must be positive, got v = -0.748811356849042. Reconsider mu_target and cv_target choices.
FAILED: mu=1.0, t_ac=0.5011872336272722, cv=0.5657591401470917 - Error: Invalid parameters: v = (mu_target * cv_target ** 2) - 1 must be positive, got v = -0.6799165953400235. Reconsider mu_target and cv_target choices.
FAILED: mu=1.0, t_ac=0.5011872336272722, cv=0.6386503549649856 - Error: Invalid parameters: v = (mu_target * cv_target ** 2) - 1 must be positive, got v = -0.5921257241030979. Reconsider mu_target and cv_target choices.
FAILED: mu=1.0, t_ac=0.5011872336272722, cv=0.7209327202223526 - Error: Invalid parameters: v = (mu_target * cv_target ** 2) - 1 must be positive, got v = -0.4802560129127992. Reconsider mu_target and cv_target choices.
FAILED: mu=1.0, t_ac=0.5011872336272722, cv=0.8138161719423083 - Error: Invalid parameters: v = (mu_target * cv_target **

In [16]:
# print success rate 
print(f"\n=== Success Rate ===")
print(f"Total combinations: {success_data['total_combo_count']}")
print(f"Successful runs: {success_data['success_count']}")
print(f"Failed runs: {success_data['failure_count']}")
print(f"Success rate: {100*success_data['success_count']/success_data['total_combo_count']:.1f}%")


=== Success Rate ===
Total combinations: 2000
Successful runs: 1577
Failed runs: 423
Success rate: 78.8%


## Test Case 2: Check against simulated data

In [18]:
# check the specified stats
from simulation.simulate_telegraph_model import simulate_one_telegraph_model_system
from stats.mean import calculate_mean
from stats.variance import calculate_variance
from stats.cv import calculate_cv
from stats.autocorrelation import calculate_autocorrelation, calculate_ac_time_interp1d

mu_target = 20
t_ac_target = 10
cv_target = 3
sigma_sum = 1

# get the parameters using the scaled equations
rho,  d, sigma_b, sigma_u = _solve_tilda_parameters(sigma_sum, mu_target, t_ac_target, cv_target)
print('Parameters found:', rho,  d, sigma_b, sigma_u)

parameter_set = [{'sigma_b':sigma_b,
                  'sigma_u':sigma_u,
                  'rho':rho,
                  'd':d,
                  'label':0
                  }]

time_points = np.arange(0, 144.0, 1.0)
size = 200
df_results = simulate_one_telegraph_model_system(parameter_set, time_points, size)
# Extract normal trajectories (remove label column and convert to numpy array)
trajectories = df_results[df_results['label'] == 0].drop('label', axis=1).values

# Extract mean and variance from the report
mean_observed = calculate_mean(trajectories, parameter_set, use_steady_state=False)
variance_observed = calculate_variance(trajectories, parameter_set, use_steady_state=False)

# Calculate CV using calculate_cv function
cv_observed = calculate_cv(variance_observed, mean_observed)

# Calculate autocorrelation for the trajectories
autocorr_results = calculate_autocorrelation(df_results)

# Get mean autocorrelation values and lags for stress condition (label=0 in this case)
ac_mean = autocorr_results['stress_ac'].mean(axis=0)  # stress_ac corresponds to label=0
lags = autocorr_results['stress_lags']

# Calculate autocorrelation time using interpolation
ac_time_observed = calculate_ac_time_interp1d(ac_mean, lags)

print(f"\n=== Observed Statistics vs Targets ===")
print(f"Mean: Target = {mu_target}, Observed = {mean_observed:.3f}")
print(f"CV: Target = {cv_target}, Observed = {cv_observed:.3f}")
print(f"AC Time: Target = {t_ac_target}, Observed = {ac_time_observed:.3f}")
print(f"Variance: Observed = {variance_observed:.3f}")

Parameters found: 201.24644268690702 0.11179116928093959 0.011109877798422587 0.9888901222015773


Simulating Telegraph Model Systems:   0%|          | 0/1 [00:00<?, ?it/s]

Running simulations on 12 cores...
System 1 parameters: {'sigma_b': 0.011109877798422587, 'sigma_u': 0.9888901222015773, 'rho': 201.24644268690702, 'd': 0.11179116928093959, 'label': 0}


Simulating Telegraph Model Systems: 100%|██████████| 1/1 [00:03<00:00,  3.99s/it]


=== Observed Statistics vs Targets ===
Mean: Target = 20, Observed = 28.290
CV: Target = 3, Observed = 2.499
AC Time: Target = 10, Observed = 11.341
Variance: Observed = 4997.661



/home/ianyang/stochastic_simulations/micromamba/envs/stochastic_sim/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/ianyang/stochastic_simulations/micromamba/envs/stochastic_sim/lib/python3.11/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


Do this for a range of parameters

In [3]:
mu_target = np.logspace(0, 4, 10)  # 1 to 10000
t_ac_target = np.logspace(-0.3, 2, 10)  # ~0.5 to 100
cv_target = np.logspace(-0.3, 0.7, 20)  # ~0.5 to 5
sigma_sum = 1
# store the relative error for plotting later
error_data = {
    # 'sigma_sum': [],
    'mean_error': [],
    'cv_error': [],
    'ac_error': [],
    'mu': [],
    't_ac': [],
    'cv': []
}

success_data = {
    'success_count':0,
    'failure_count':0,
    'total_combo_count':len(mu_target) * len(t_ac_target) * len(cv_target)
}

# loop through every combination
for mu in mu_target:
    for t_ac in t_ac_target:
        for cv in cv_target:
            # get the parameters using the scaled equations
            try:
                rho,  d, sigma_b, sigma_u = _solve_tilda_parameters(sigma_sum, mu, t_ac, cv,
                                                                #   check_biological=False
                                                                  )
                success_data['success_count'] += 1
                
                print(f"Testing with mu={mu}, t_ac={t_ac}, cv={cv}")
                print('Parameters found:', rho,  d, sigma_b, sigma_u)

                
                parameter_set = [{'sigma_b':sigma_b,
                                'sigma_u':sigma_u,
                                'rho':rho,
                                'd':d,
                                'label':0
                                }]

                time_points = np.arange(0, 
                                        int(t_ac * 20),
                                         1.0)
                size = 200
                df_results = simulate_one_telegraph_model_system(parameter_set, time_points, size)
                # Extract normal trajectories (remove label column and convert to numpy array)
                trajectories = df_results[df_results['label'] == 0].drop('label', axis=1).values

                # Extract mean and variance from the report
                mean_observed = calculate_mean(trajectories, parameter_set, use_steady_state=False)
                variance_observed = calculate_variance(trajectories, parameter_set, use_steady_state=False)

                # Calculate CV using calculate_cv function
                cv_observed = calculate_cv(variance_observed, mean_observed)

                # Calculate autocorrelation for the trajectories
                autocorr_results = calculate_autocorrelation(df_results)

                # Get mean autocorrelation values and lags for stress condition (label=0 in this case)
                ac_mean = autocorr_results['stress_ac'].mean(axis=0)  # stress_ac corresponds to label=0
                lags = autocorr_results['stress_lags']

                # Calculate autocorrelation time using interpolation
                ac_time_observed = calculate_ac_time_interp1d(ac_mean, lags)

                print(f"\n=== Observed Statistics vs Targets ===")
                print(f"Mean: Target = {mu}, Observed = {mean_observed:.3f}")
                print(f"CV: Target = {cv}, Observed = {cv_observed:.3f}")
                print(f"AC Time: Target = {t_ac}, Observed = {ac_time_observed:.3f}")
                print(f"Variance: Observed = {variance_observed:.3f}")
                
                mean_rel_error = abs(mean_observed - mu) / mu
                cv_rel_error = abs(cv_observed - cv) / cv  
                ac_rel_error = abs(ac_time_observed - t_ac) / t_ac
                
                # Store errors for analysis
                error_data['mean_error'].append(mean_rel_error * 100)  # Convert to percentage
                error_data['cv_error'].append(cv_rel_error * 100)
                error_data['ac_error'].append(ac_rel_error * 100)
                # Check tolerances with informative error messages
                assert mean_rel_error < 0.2, f"❌Mean error too large: {mean_rel_error:.3f} (target: {mu}, observed: {mean_observed:.3f})"
                assert cv_rel_error < 0.2, f"❌CV error too large: {cv_rel_error:.3f} (target: {cv}, observed: {cv_observed:.3f})"
                assert ac_rel_error < 0.2, f"❌AC time error too large: {ac_rel_error:.3f} (target: {t_ac}, observed: {ac_time_observed:.3f})"
                
                print(f"✅ All assertions passed - Mean: {mean_rel_error:.1%}, CV: {cv_rel_error:.1%}, AC: {ac_rel_error:.1%}")  
                
            except Exception as e:
                success_data['failure_count'] += 1
                print(f"FAILED: mu={mu}, t_ac={t_ac}, cv={cv} - Error: {e}")
                # Removed the 'break' statement - now continues to next combination
            except AssertionError as e:
                print(f"❌ ASSERTION FAILED: {e}")
                success_data['failure_count'] += 1
                continue    

FAILED: mu=1.0, t_ac=0.5011872336272722, cv=0.5011872336272722 - Error: Invalid parameters: v = (mu_target * cv_target ** 2) - 1 must be positive, got v = -0.748811356849042. Reconsider mu_target and cv_target choices.
FAILED: mu=1.0, t_ac=0.5011872336272722, cv=0.5657591401470917 - Error: Invalid parameters: v = (mu_target * cv_target ** 2) - 1 must be positive, got v = -0.6799165953400235. Reconsider mu_target and cv_target choices.
FAILED: mu=1.0, t_ac=0.5011872336272722, cv=0.6386503549649856 - Error: Invalid parameters: v = (mu_target * cv_target ** 2) - 1 must be positive, got v = -0.5921257241030979. Reconsider mu_target and cv_target choices.
FAILED: mu=1.0, t_ac=0.5011872336272722, cv=0.7209327202223526 - Error: Invalid parameters: v = (mu_target * cv_target ** 2) - 1 must be positive, got v = -0.4802560129127992. Reconsider mu_target and cv_target choices.
FAILED: mu=1.0, t_ac=0.5011872336272722, cv=0.8138161719423083 - Error: Invalid parameters: v = (mu_target * cv_target **

Simulating Telegraph Model Systems:   0%|          | 0/1 [00:00<?, ?it/s]

Simulating Telegraph Model Systems: 100%|██████████| 1/1 [00:00<00:00,  5.37it/s]
/home/ianyang/stochastic_simulations/micromamba/envs/stochastic_sim/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/ianyang/stochastic_simulations/micromamba/envs/stochastic_sim/lib/python3.11/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


Running simulations on 12 cores...
System 1 parameters: {'sigma_b': 0.9010868221386558, 'sigma_u': 0.09891317786134428, 'rho': 2.436795260012077, 'd': 2.1957640970468217, 'label': 0}

=== Observed Statistics vs Targets ===
Mean: Target = 1.0, Observed = 0.868
CV: Target = 1.037025590866764, Observed = 1.162
AC Time: Target = 0.5011872336272722, Observed = 0.733
Variance: Observed = 1.018
FAILED: mu=1.0, t_ac=0.5011872336272722, cv=1.037025590866764 - Error: ❌AC time error too large: 0.463 (target: 0.5011872336272722, observed: 0.733)
Testing with mu=1.0, t_ac=0.5011872336272722, cv=1.1706337816171068
Parameters found: 4.633018301259153 3.110541687097266 0.6713855816740222 0.3286144183259778


Simulating Telegraph Model Systems: 100%|██████████| 1/1 [00:00<00:00,  4.19it/s]
/home/ianyang/stochastic_simulations/micromamba/envs/stochastic_sim/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/ianyang/stochastic_simulations/micromamba/envs/stochastic_sim/lib/python3.11/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


...


=== Observed Statistics vs Targets ===
Mean: Target = 2.7825594022071245, Observed = 2.760
CV: Target = 1.1706337816171068, Observed = 1.166
AC Time: Target = 17.1132830416178, Observed = 16.077
Variance: Observed = 10.350
✅ All assertions passed - Mean: 0.8%, CV: 0.4%, AC: 6.1%
Testing with mu=2.7825594022071245, t_ac=17.1132830416178, cv=1.3214557699755294
Parameters found: 4.266781593192334 0.06139344942572311 0.040037418415330844 0.9599625815846693


Simulating Telegraph Model Systems:   0%|          | 0/1 [00:00<?, ?it/s]

Running simulations on 12 cores...
System 1 parameters: {'sigma_b': 0.040037418415330844, 'sigma_u': 0.9599625815846693, 'rho': 4.266781593192334, 'd': 0.06139344942572311, 'label': 0}


Simulating Telegraph Model Systems:   0%|          | 0/1 [00:00<?, ?it/s]


KeyboardInterrupt: 